In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.layers import Dense , SimpleRNN, GRU,LSTM, Flatten ,Dropout, Input, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
import re
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
text= open("/content/computer science.txt", encoding='utf-8')
text=text.read()

In [5]:
text = re.sub('[^a-zA-Z.,]',' ', text).lower()
sents=sent_tokenize(text)

In [6]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

In [7]:

words=word_tokenize(text.lower())
len(words)

32541

In [8]:

unique =set(words)
unique =list(unique)


In [9]:
len(unique)

4953

In [10]:
data=[]
for i in words:
  if i in wv.vocab.keys():
    data.append(wv[i])
  else:
    data.append(np.zeros(300))

data=np.array(data)


In [11]:
x=[]
y=[]
for i in range (data.shape[0]):
  if data.shape[0]<i+6:
    break
  x.append(np.add(data[i+3],data[i+4],np.add(data[i],data[i+1],data[i+2])))
  y.append(unique.index(words[i+5]))

  

In [12]:
len(x),len(y)

(32536, 32536)

In [13]:
x=np.array(x)
y=to_categorical(y)
x.shape,y.shape

((32536, 300), (32536, 4953))

In [14]:
x_train,x_test ,y_train,y_test =train_test_split(x,y,test_size=.03,shuffle=True)

In [15]:
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)


In [16]:
x_train.shape

(31559, 300)

In [17]:
input_shape = ( 300,1)
batch_size = 256
units = 100
dropout = 0.2

In [18]:
model = Sequential()
model.add(LSTM(units=units,
                    dropout=dropout,
                    input_shape=input_shape))
model.add(Dense(4953))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 4953)              500253    
                                                                 
 activation (Activation)     (None, 4953)              0         
                                                                 
Total params: 541,053
Trainable params: 541,053
Non-trainable params: 0
_________________________________________________________________


In [19]:
# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='rnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer
# accuracy is good metric for classification tasks
#model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss=tf.keras.losses.CategoricalCrossentropy())
# train the network
model.fit(x_train, y_train, epochs=300, batch_size=batch_size,)


Epoch 1/300
124/124 [==============================] - 10s 28ms/step - loss: 6.9843
Epoch 2/300
124/124 [==============================] - 3s 25ms/step - loss: 6.4444
Epoch 3/300
124/124 [==============================] - 3s 25ms/step - loss: 6.4276
Epoch 4/300
124/124 [==============================] - 3s 25ms/step - loss: 6.4220
Epoch 5/300
124/124 [==============================] - 3s 27ms/step - loss: 6.4179
Epoch 6/300
124/124 [==============================] - 3s 25ms/step - loss: 6.4159
Epoch 7/300
124/124 [==============================] - 3s 25ms/step - loss: 6.4125
Epoch 8/300
124/124 [==============================] - 3s 25ms/step - loss: 6.4104
Epoch 9/300
124/124 [==============================] - 3s 27ms/step - loss: 6.4080
Epoch 10/300
124/124 [==============================] - 3s 26ms/step - loss: 6.4058
Epoch 11/300
124/124 [==============================] - 3s 26ms/step - loss: 6.4039
Epoch 12/300
124/124 [==============================] - 3s 26ms/step - loss: 6.4036


In [20]:
x_test.shape

(977, 300)

In [21]:

pred=model.predict(x_test.reshape(-1,300,1))
from sklearn.metrics import accuracy_score
y_true=[ np.argmax(z) for z in y_test]
y_pred= [np.argmax(x) for x in pred]
acc=accuracy_score(y_pred,y_true)

31/31 [==============================] - 1s 7ms/step


In [22]:
acc*100

11.975435005117706